## Overview

This script will move downloaded images to `prepare_dataset`. The process is briefed as follows:

- Images downloaded from `notebooks/download.ipynb` will be stored in `download_file` folder.
- Under `download_file` folder, images will be grouped according to the polygon they belong to.
- In each polygon, 2 types of images are present: geotiff with band information, and a mask file.
- Geotiffs should be moved and stored in `prepare_dataset/images_directory{group_id}` folder.
- Mask files are stored in `prepare_dataset/mask_directory{group_id}` folder.

## Implementation

In [17]:
import os
import glob
import shutil
import pandas as pd


### Get all subfolders

In [18]:
def fast_scandir(dirname: str) -> list:
    """
    Scan and return all subfolders of a directory.
    """
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

source_path = "../download_file2/"
subfolders_list = fast_scandir(source_path)
print(f"Number of subfolders: {len(subfolders_list)}")

for folder in subfolders_list:
    print(folder)


Number of subfolders: 1327
../download_file2/S2A_10UEA_20230813_0_L2A
../download_file2/S2A_10UEB_20230816_0_L2A
../download_file2/S2A_10UEC_20230816_0_L2A
../download_file2/S2A_10UFA_20230813_0_L2A
../download_file2/S2A_10UFV_20230813_0_L2A
../download_file2/S2A_11ULT_20230820_0_L2A
../download_file2/S2B_10UEA_20230815_0_L2A
../download_file2/S2B_10UEC_20230828_0_L2A
../download_file2/S2B_10UFB_20230828_0_L2A
../download_file2/S2B_10UFC_20230828_0_L2A
../download_file2/S2B_10UFV_20230815_0_L2A
../download_file2/S2B_10UGA_20230815_0_L2A
../download_file2/S2B_10UGB_20230828_0_L2A
../download_file2/S2B_10UGC_20230828_0_L2A
../download_file2/S2B_10UGD_20230828_0_L2A
../download_file2/S2B_11ULR_20230815_0_L2A
../download_file2/S2B_11ULU_20230828_0_L2A
../download_file2/S2A_10UEA_20230813_0_L2A\20230813
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\0_10
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\0_11
../dow

### Get folders with restrictions - Regex

In [19]:
def get_subfolders_with_keyword(keyword: str, subfolders_list: list = subfolders_list) -> list:
    subfolders_with_keyword_list = []

    for folder in subfolders_list:
        if keyword in folder:
            subfolders_with_keyword_list.append(folder)

    return subfolders_with_keyword_list

subfolders_with_keyword_list = get_subfolders_with_keyword("tiles/") # note that we need the / to get folders
for folder in subfolders_with_keyword_list:
    print(folder)


### Get folders with restrictions - DataFrame

#### 1. Turn paths into columns

In [20]:
raw_df = pd.DataFrame(subfolders_list, columns=["path_name"])
raw_df.head()


,path_name
0,../download_file2/S2A_10UEA_20230813_0_L2A
1,../download_file2/S2A_10UEB_20230816_0_L2A
2,../download_file2/S2A_10UEC_20230816_0_L2A
3,../download_file2/S2A_10UFA_20230813_0_L2A
4,../download_file2/S2A_10UFV_20230813_0_L2A


In [21]:
test_df = pd.DataFrame([x.rsplit('/') for x in raw_df['path_name']])
test_df


,0,1,2
0,..,download_file2,S2A_10UEA_20230813_0_L2A
1,..,download_file2,S2A_10UEB_20230816_0_L2A
2,..,download_file2,S2A_10UEC_20230816_0_L2A
3,..,download_file2,S2A_10UFA_20230813_0_L2A
4,..,download_file2,S2A_10UFV_20230813_0_L2A
...,...,...,...
1322,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_5
1323,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_6
1324,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_7
1325,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_8


#### 2. Filter out rows that have `tile` as part of the path names

In [22]:
df_subfolder_under_tiles = test_df[(test_df[2].str.contains("tiles")) & (test_df[2].notnull())].reset_index(drop=True)
df_subfolder_under_tiles


,0,1,2
0,..,download_file2,S2A_10UEA_20230813_0_L2A\20230813\tiles
1,..,download_file2,S2A_10UEA_20230813_0_L2A\20230813\tiles\0_10
2,..,download_file2,S2A_10UEA_20230813_0_L2A\20230813\tiles\0_11
3,..,download_file2,S2A_10UEA_20230813_0_L2A\20230813\tiles\0_12
4,..,download_file2,S2A_10UEA_20230813_0_L2A\20230813\tiles\0_13
...,...,...,...
1288,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_5
1289,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_6
1290,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_7
1291,..,download_file2,S2B_11ULU_20230828_0_L2A\20230828\tiles\9_8


#### 3. Reconcatenate to pathnames and indicate date

In [25]:
df_selected_paths_date = df_subfolder_under_tiles
df_selected_paths_date["file_path"] = df_selected_paths_date[[0, 1, 2]].apply(lambda x: "/".join(x), axis=1)
df_selected_paths_date["date"] = df_selected_paths_date[2].apply(lambda x: pd.to_datetime(x, format="%Y%m%d"))
df_selected_paths_date.drop([0, 1, 2], axis=1, inplace=True)
df_selected_paths_date


ValueError: time data "S2A_10UEA_20230813_0_L2A\20230813\tiles" doesn't match format "%Y%m%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

#### Save and copy selected file paths to a list

In [28]:
selected_path_list = df_selected_paths_date["file_path"].values.tolist()

for path in selected_path_list:
    print(path)


../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\0_10
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\0_11
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\0_12
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\0_13
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\1_10
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\1_11
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\1_12
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\1_13
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\1_8
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\1_9
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\2_10
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\2_11
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\2_12
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\2_13
../download_file2/S2A_10UEA_20230813_0_L2A\20230813\tiles\2_7


### Given a list of directories, get a list of files inside, and move to the selected directory

For now, I will use the shorter list - `subfolders_with_keyword_list`.

In [29]:
def get_list_of_files_in_directory(directory_name: str, keyword: str = ".tif") -> list:
    return [f"{directory_name}/{f}" for f in os.listdir(directory_name) if f.endswith(keyword)]

test_get_list_of_files_in_directory = get_list_of_files_in_directory(subfolders_with_keyword_list[0])
print(test_get_list_of_files_in_directory)


IndexError: list index out of range

For now, I will use the shorter list - `subfolders_with_keyword_list`.

In [30]:
def move_file(source_path: str, label: str, id: int) -> None:
    """
    label: either "image" or "mask"
    the moved file will be named as `date_original-name`
    """
    file_name = source_path.split("/")[-1]
    file_date = source_path.split("/")[-4]
    destination_folder = f"../prepare_dataset/{label}_directory{id}"
    destination_path = f"../prepare_dataset/{label}_directory{id}/{file_date}_{file_name}"

    if not os.path.isdir(destination_folder):
        os.makedirs(os.path.dirname(destination_path))

    if os.path.isfile(destination_path):
        print("File exists.")
        return

    shutil.copy(source_path, destination_path)
    print(f"File copied to destination: {destination_path}.")

test_move_file = move_file(test_get_list_of_files_in_directory[1], "image", 1)


NameError: name 'test_get_list_of_files_in_directory' is not defined

#### Batch move files

There are two things we need to do:

1. Construct a set so that geotif under the same folder gets the same `id` when `move_file()` gets executed.
2. `mask.tif` should get the label `mask`, others get `image`.

In [23]:
def batch_move_files(source_path_list: list) -> None:
    path_dict = {}

    for i in range(len(source_path_list)):
        current_path = source_path_list[i]
        current_folder = current_path.rsplit("/", 1)[0] # split on the last occurrence

        if current_folder not in path_dict:
            path_dict[current_folder] = len(path_dict)

        current_id = path_dict[current_folder]

        if "mask" in current_path:
            move_file(current_path, "mask", current_id)
        else:
            print(f"image: {current_path}")
            move_file(current_path, "image", current_id)

batch_move_files(test_get_list_of_files_in_directory)


image: ../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_4/B08.tif
File copied to destination: ../prepare_dataset/image_directory0/20230820_B08.tif.
image: ../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_4/B09.tif
File copied to destination: ../prepare_dataset/image_directory0/20230820_B09.tif.
image: ../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_4/B8A.tif
File copied to destination: ../prepare_dataset/image_directory0/20230820_B8A.tif.
File copied to destination: ../prepare_dataset/mask_directory0/20230820_mask.tif.
image: ../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_4/B02.tif
File copied to destination: ../prepare_dataset/image_directory0/20230820_B02.tif.
image: ../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_4/B03.tif
File copied to destination: ../prepare_dataset/image_directory0/20230820_B03.tif.
image: ../download_file/S2A_7VFG_20230820_0_L2A/20230820/tiles/9_4/B01.tif
File copied to destination: ../prepare_dataset/image_direc